# MEx 1: The Julia framework for HPC

## **OBJECTIVE**: Explore Julia framework for HPC
- [ ] KR1: Ran different versions of `@code_[mode]` to examine a simple expression (replace `[mode]` with `native`, `typed`, `warntype`, etc.
- [ ] KR2: Showed that constants in codes are passed on after constant function or expression evaluations. May have gone beyond replicating those done in class.
- [ ] KR3: Created two versions of the same function such as `pos(x)` in the book, one with type-instability and the other fixed.
- [ ] KR4: Demonstrated Julia's type-inference and multiple dispatch using the `iam()` function sample as discussed in class. (See Chapter 3)
- [ ] KR5: Ran a function `poly(A,x)` that evaluates the $N$-th order polynomial via a naive approach.
- [ ] KR6: Evaluated the same polynomial using [the package `Polynomials` in Julia](https://juliamath.github.io/Polynomials.jl/stable/).
- [ ] KR7. Used `BenchmarkTools` to plot timing distribution and obtain the best time via the macro `@btime` evaluated at some random number. (See Chapter 2)
- [ ] KR8 (Bonus): Plotted of the time $T$ it takes for the function to run using `@time` macro for different polynomial lengths $N$.

[Full documentation is found here.](./README.md)